In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightning as L
from torch.utils.data import DataLoader, TensorDataset


In [2]:
class LSTMbyHand(L.LightningModule):
    def __init__(self):
        super().__init__()
        
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)

        self.wlr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
        self.wpr1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

        self.wp1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

        self.wo1 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean = mean, std = std), requires_grad=True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad=True)


    def lstm_unit(self, input_value, long_memory, short_memory):
        long_remember_percent = torch.sigmoid((short_memory * self.wlr1) 
                                              + (input_value * self.wlr2)
                                              + self.blr1)
        
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1)
                                                    + (input_value * self.wpr2)
                                                    + self.bpr1)
        
        potential_memory = torch.tanh((short_memory * self.wp1)
                                      + (input_value * self.wp2)
                                      + self.bp1)

        updated_long_memory = ((long_memory * long_remember_percent) + (potential_memory * potential_remember_percent))

        output_percent = torch.sigmoid((short_memory * self.wo1)
                                       + (input_value * self.wo2)
                                       + self.bo1)
        
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent

        return ([updated_long_memory, updated_short_memory])


    def forward(self, input):
        
        long_memory = 0
        short_memory = 0
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]

        long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)

        return short_memory
    
    def configure_optimizers(self):
    
        return Adam(self.parameters())
    
    def training_step(self, batch, batch_idx):

        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i)**2

        self.log('train_loss', loss)

        if (label_i == 0):
            self.log("out_0",output_i)
        else:
            self.log("out_1",output_i)

        return loss


In [3]:

model = LSTMbyHand()

print("Company A: Observer = 0 , Predicted = ", model(torch.tensor([0.,0.5,0.25,1.])).detach())

print("Company B: Observer = 1 , Predicted = ", model(torch.tensor([1,0.5,0.25,1.])).detach())


Company A: Observer = 0 , Predicted =  tensor(-0.3238)
Company B: Observer = 1 , Predicted =  tensor(-0.3531)


In [4]:

inputs = torch.tensor([[0.,0.5,0.25,1.],[1,0.5,0.25,1.]])

labels = torch.tensor([0.,1.])

dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

trainer = L.Trainer(max_epochs=2000)
trainer.fit(model, train_dataloaders=dataloader)

print("Company A: Observer = 0 , Predicted = ", model(torch.tensor([0.,0.5,0.25,1.])).detach())

print("Company B: Observer = 1 , Predicted = ", model(torch.tensor([1,0.5,0.25,1.])).detach())

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
2025-11-23 13:30:00.428927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
0         Modules in eval mode
/workspaces/stock_predictor_lstm_model/ven

Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 130.67it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=2000` reached.


Epoch 1999: 100%|██████████| 2/2 [00:00<00:00, 96.42it/s, v_num=0] 
Company A: Observer = 0 , Predicted =  tensor(0.5032)
Company B: Observer = 1 , Predicted =  tensor(0.5148)


In [5]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

trainer = L.Trainer(max_epochs=4000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

print("Company A: Observer = 0 , Predicted = ", model(torch.tensor([0.,0.5,0.25,1.])).detach())

print("Company B: Observer = 1 , Predicted = ", model(torch.tensor([1,0.5,0.25,1.])).detach())


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
Restoring states from the checkpoint path at /workspaces/stock_predictor_lstm_model/notebooks/lightning_logs/version_0/checkpoints/epoch=1999-step=4000.ckpt
/workspaces/stock_predictor_lstm_model/venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:445: The dirpath has changed from '/workspaces/stock_predictor_lstm_model/notebooks/lightning_logs/version_0/checkpoints' to '/workspaces/stock_predictor_lstm_model/notebooks/lightning_logs/version_1/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
-----------------------

Epoch 3999: 100%|██████████| 2/2 [00:00<00:00, 136.29it/s, v_num=1]

`Trainer.fit` stopped: `max_epochs=4000` reached.


Epoch 3999: 100%|██████████| 2/2 [00:00<00:00, 99.67it/s, v_num=1] 
Company A: Observer = 0 , Predicted =  tensor(0.4892)
Company B: Observer = 1 , Predicted =  tensor(0.5313)


In [6]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

trainer = L.Trainer(max_epochs=5000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

print("Company A: Observer = 0 , Predicted = ", model(torch.tensor([0.,0.5,0.25,1.])).detach())

print("Company B: Observer = 1 , Predicted = ", model(torch.tensor([1,0.5,0.25,1.])).detach())


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
Restoring states from the checkpoint path at /workspaces/stock_predictor_lstm_model/notebooks/lightning_logs/version_1/checkpoints/epoch=3999-step=8000.ckpt
/workspaces/stock_predictor_lstm_model/venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:445: The dirpath has changed from '/workspaces/stock_predictor_lstm_model/notebooks/lightning_logs/version_1/checkpoints' to '/workspaces/stock_predictor_lstm_model/notebooks/lightning_logs/version_2/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
-----------------------

Epoch 4999: 100%|██████████| 2/2 [00:00<00:00, 114.86it/s, v_num=2]

`Trainer.fit` stopped: `max_epochs=5000` reached.


Epoch 4999: 100%|██████████| 2/2 [00:00<00:00, 85.33it/s, v_num=2] 
Company A: Observer = 0 , Predicted =  tensor(0.4440)
Company B: Observer = 1 , Predicted =  tensor(0.5764)
